In [3]:
import redis
import time

In [4]:
pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
client = redis.Redis(connection_pool=pool)

### Key:value 형태로 저장하기

In [5]:
client.set("test", 1234)

True

In [6]:
client.get("test")

b'1234'

client.flushall() 명령어로 데이터베이스를 모두 삭제할 수 있습니다.

In [7]:
client.flushall()

True

In [8]:
print(client.get("test"))

None


다음과 같이 ex인자를 전달하여 데이터가 저장되는 시간에 제한을 둘 수 있습니다.

In [9]:
client.set("expire", "I'm here", ex=1) # 1초 후 만료
print(client.get("expire"))

time.sleep(1)
print(client.get("expire"))

b"I'm here"
None


client.expire 메서드를 이용하여 만료시간을 재설정해줄 수 있습니다.

In [10]:
client.set("expire", "I'm here", ex=1)
print(client.get("expire"))
client.expire("expire", 3)

time.sleep(1)
print(client.get("expire"))

b"I'm here"
b"I'm here"


client.ttl() 메서드를 이용하여 만료까지 남은 시간을 측정할 수 있습니다.

In [11]:
client.set("expire", "I'm here", ex=10)
for _ in range(3):
    time.sleep(1)
    print(client.ttl("expire"))

9
8
7


### Hashs

Hashs는 하나의 Hash키 아래에 또다시 여러개의 Key:value쌍 데이터를 저장할 수 있습니다.

In [12]:
user1 = {"name": "eva",
         "age": 41,
         "gender": "female",
         "country": "France"}

다음과 같은 방법으로 읽고 쓸 수 있습니다.

```python
client.hset(Name, Key, Value)
```

In [13]:
client.hset("user1", "name", "eva")

1

In [14]:
client.hget("user1", "name")

b'eva'

In [15]:
client.hset("user1", "age", 41)

1

In [16]:
client.hget("user1", "age")

b'41'

여러개의 Key:value 쌍을 한번에 저장할 수도 있습니다.

In [17]:
client.hmset("user1", user1)

/tmp/ipykernel_11425/1155743130.py:1: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  client.hmset("user1", user1)


True

In [18]:
client.hmget("user1", "name", "age", "gender", "country")

[b'eva', b'41', b'female', b'France']

하지만, DeprecationWarning이 출력됩니다. 앞으로는 hset()을 이용하라고 하네요. 다음의 방법으로 hmset의 기능을 동일하게 이용할 수 있습니다. 저장된 데이터쌍의 개수를 Return합니다.

In [19]:
client.flushall()
client.hset("user1", mapping = user1)

4

In [20]:
client.hget("user1", "name") # Hash key 내에 있는 특정 Key의 Value 가져오기

b'eva'

In [21]:
client.hmget("user1", "name", "age") # Hash Key 내에 있는 일부 Key들의 value 가져오기

[b'eva', b'41']

In [22]:
client.hgetall("user1") # Hash key에 저장된 모든 Key:value 가져오기

{b'name': b'eva', b'age': b'41', b'gender': b'female', b'country': b'France'}

그 외 다음과 같은 Hashs와 관련된 메서드들이 존재합니다.

In [23]:
client.hkeys("user1") # Hash key에 저장된 모든 Key 가져오기

[b'name', b'age', b'gender', b'country']

In [24]:
client.hvals("user1") # Hash key에 저장된 모든 value 가져오기

[b'eva', b'41', b'female', b'France']

In [25]:
client.hlen("user1") # Hash key에 저장된 Key:value의 개수 조회

4

In [26]:
client.hexists("user1", "age") # key가 있는지 확인

True

In [27]:
client.hexists("user1", "email")

False

In [28]:
client.hsetnx("user1", "name", "eva") # Key가 DB에 없으면 저장

0

In [29]:
client.hsetnx("user1", "email", "eva123@gmail.com")

1

In [30]:
client.delete("user1") # 특정 Hash Key와 저장된 데이터 삭제하기

1

In [31]:
client.hgetall("user1")

{}